# Generating files with label for leaderboard test

In this kit, we will show how to use the trained model to generate labels and save the labels for uploading. We recommand you to read the 'LeaderboardDataGuide' to get familar with the test data.

In [1]:
from util.preproc import balance_set,plot_confusion_matrix
import torch
import numpy as np
from numpy.random import RandomState
import pickle
import matplotlib.pyplot as plt 
from braindecode.util import np_to_var, var_to_np
import torch.nn.functional as F
from torch import nn
from braindecode.util import set_random_seeds
import util.shallow_net
from util.utilfunc import get_balanced_batches


cuda = torch.cuda.is_available()
print('gpu condition: ', cuda)
device = 'cuda' if cuda else 'cpu'

SEED=42
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
np.random.seed(SEED)
torch.backends.cudnn.deterministic = True
rng = RandomState(SEED)

gpu condition:  True


**These Objects are the same as in sleep and motor imagery kits.**

In [2]:
class TestObject(object):
    def __init__(self, X):
        mean=np.mean(X,axis=1,keepdims=True)
        std=np.std(X,axis=1,keepdims=True)
        X=(X-mean)/(std)
        #we scale it to 1000 as a better training scale of the shallow CNN
        #according to the orignal work of the paper
        self.X = X.astype(np.float32)*1e3
class SleepObject(object):
    def __init__(self, X):
        mean=np.mean(X,axis=2,keepdims=True)
        #here normalise across the window, when channel size is large enough
        std=np.std(X,axis=2,keepdims=True)
        X=(X-mean)/(std)
        #we scale it to 1000 as a better training scale of the shallow CNN
        #according to the orignal work of the paper
        self.X = X.astype(np.float32)*1e3

In [3]:
sleep_label=None
MI_s1=None
MI_s2=None
MI_s3=None
MI_s4=None
MI_s5=None

**Loading test data has been explained in the DataGuide. Replace the 'savebase' part with your folder. After loading the test data, model trained with the training set can be loaded to do the prediction. 13094 trials for leaderboard sleep test data.**

In [6]:
savebase = '/home/moja/mne_data/MNE-beetlsleepleaderboard-data/testing/'

X_test = []
for subj in range(6, 18):
    for session in range(1, 3):
        with open(savebase + "leaderboard_s{}r{}X.npy".format(subj, session), 'rb') as f:
            X_test.append(pickle.load(f))
X_test = np.concatenate(X_test)

print('overall test size')
print(X_test.shape)
test_set = SleepObject(X_test)
print(test_set.X[1,:,:20])


savebase0='D:\\sleep'

set_random_seeds(seed=42, cuda=cuda)
input_time_length = X_test.shape[2]
print(input_time_length)
in_chans=X_test.shape[1]
model = util.shallow_net.EEGShallowClassifier(in_chans, 6, input_time_length, return_feature=False)
if cuda:
    model.cuda()

checkpoint = torch.load(savebase0+'\\cnn_model_sleep.pth')
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

average_acc=[]
average_loss=[]
setname='testset'
dataset=test_set

#shuffle=False to make sure it's in the orignial order
i_trials_in_batch = get_balanced_batches(len(dataset.X), rng, shuffle=False,
                                    batch_size=30)
outputs=None
for i_trials in i_trials_in_batch:
    # Have to add empty fourth dimension to X
    batch_X = dataset.X[i_trials][:,:,:,None]
    net_in = np_to_var(batch_X)
    if cuda:
        net_in = net_in.cuda()
    toutputs = model(net_in)
    if outputs is None:
        temp=toutputs.cpu()
        outputs=temp.detach().numpy()
    else:
        temp=toutputs.cpu()
        outputs=np.concatenate((outputs,temp.detach().numpy()))
predicted_labels = np.argmax((outputs), axis=1)
print(predicted_labels.shape)
sleep_label = predicted_labels
predicted_labels[:100]

FileNotFoundError: [Errno 2] No such file or directory: './LeaderboardSleep/testing/leaderboard_s6r1X.npy'

**200 trials for leaderboard MI test data for each subject 1-5**

In [7]:
pilotname='S1'
savebase0='/home/moja/mne_data/MNE-beetlmileaderboard-data/'#'D:\\leaderboardData\\leaderboardMI\\'
savebase = savebase0+pilotname+'/'+'testing/'

# read crop data
ch_names =['Fp1', 'Fz', 'F3', 'F7', 'FT9', 'FC5', 'FC1', 'C3', 'T7', 
           'TP9', 'CP5', 'CP1', 'Pz', 'P3', 'P7', 'O1', 'Oz', 
           'O2', 'P4', 'P8', 'TP10', 'CP6', 'CP2', 'C4', 'T8',
           'FT10', 'FC6', 'FC2', 'F4', 'F8', 'Fp2', 'AF7', 'AF3', 
           'AFz', 'F1', 'F5', 'FT7', 'FC3', 'FCz', 'C1', 'C5', 
           'TP7', 'CP3', 'P1', 'P5', 'PO7', 'PO3', 'POz', 'PO4', 
           'PO8', 'P6', 'P2', 'CPz', 'CP4', 'TP8', 'C6', 'C2',
           'FC4', 'FT8', 'F6', 'F2', 'AF4', 'AF8']
#here we used the ones arround motor cortex
good_indices=[7,8,11,12,13,14,19,22,23,24,28]+[39,40,43,44,45,51,52,53,54,57,58]
good_indices[:] = [x - 1 for x in good_indices]

for i in range(5,15):
    with open (savebase+"race"+str(i+1)+"_padsData.npy", 'rb') as fp:
        Xt = pickle.load(fp)
    if i==5:
        X0 = Xt
    else:
        X0 = np.concatenate((X0,Xt))
    print('run',i+1)
    print(X0.shape)   
    
X_test = X0[:,good_indices,:]

print('overall test size')
print(X_test.shape)
test_set = TestObject(X_test)

print(cuda)
set_random_seeds(seed=42, cuda=cuda)
input_time_length = X_test.shape[2]
print(input_time_length)
in_chans=X_test.shape[1]
labelsize=3
model = util.shallow_net.EEGShallowClassifier(in_chans, labelsize, input_time_length, return_feature=False)
if cuda:
    model.cuda()

checkpoint = torch.load('D:\\motor_imagery\\MImodelinLeaderboard\\'+'cnn_model_MI_'+pilotname+'.pth')
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

average_acc=[]
average_loss=[]
setname='testset'
dataset=test_set

i_trials_in_batch = get_balanced_batches(len(dataset.X), rng, shuffle=False,
                                    batch_size=30)
outputs=None
for i_trials in i_trials_in_batch:
    # Have to add empty fourth dimension to X
#             print(i_trials)
    batch_X = dataset.X[i_trials][:,:,:,None]
#     batch_y = dataset.y[i_trials]
    net_in = np_to_var(batch_X)
    if cuda:
        net_in = net_in.cuda()
    toutputs = model(net_in)
    if outputs is None:
        temp=toutputs.cpu()
        outputs=temp.detach().numpy()
    else:
        temp=toutputs.cpu()
        outputs=np.concatenate((outputs,temp.detach().numpy()))
predicted_labels = np.argmax((outputs), axis=1)
print(predicted_labels.shape)
MI_s1 = predicted_labels
predicted_labels

run 6
(20, 63, 2000)
run 7
(40, 63, 2000)
run 8
(60, 63, 2000)
run 9
(80, 63, 2000)
run 10
(100, 63, 2000)
run 11
(120, 63, 2000)
run 12
(140, 63, 2000)
run 13
(160, 63, 2000)
run 14
(180, 63, 2000)
run 15
(200, 63, 2000)
overall test size
(200, 22, 2000)
True
2000


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\motor_imagery\\MImodelinLeaderboard\\cnn_model_MI_S1.pth'

In [ ]:
pilotname='S2'
savebase0='D:\\leaderboardData\\leaderboardMI\\'
savebase = savebase0+pilotname+'\\'+'testing\\'

# read crop data
ch_names =['Fp1', 'Fz', 'F3', 'F7', 'FT9', 'FC5', 'FC1', 'C3', 'T7', 
           'TP9', 'CP5', 'CP1', 'Pz', 'P3', 'P7', 'O1', 'Oz', 
           'O2', 'P4', 'P8', 'TP10', 'CP6', 'CP2', 'C4', 'T8',
           'FT10', 'FC6', 'FC2', 'F4', 'F8', 'Fp2', 'AF7', 'AF3', 
           'AFz', 'F1', 'F5', 'FT7', 'FC3', 'FCz', 'C1', 'C5', 
           'TP7', 'CP3', 'P1', 'P5', 'PO7', 'PO3', 'POz', 'PO4', 
           'PO8', 'P6', 'P2', 'CPz', 'CP4', 'TP8', 'C6', 'C2',
           'FC4', 'FT8', 'F6', 'F2', 'AF4', 'AF8']
#here we used the ones arround motor cortex
good_indices=[7,8,11,12,13,14,19,22,23,24,28]+[39,40,43,44,45,51,52,53,54,57,58]
good_indices[:] = [x - 1 for x in good_indices]

for i in range(5,15):
    with open (savebase+"race"+str(i+1)+"_padsData.npy", 'rb') as fp:
        Xt = pickle.load(fp)
    if i==5:
        X0 = Xt
    else:
        X0 = np.concatenate((X0,Xt))
    print('run',i+1)
    print(X0.shape)   
    
X_test = X0[:,good_indices,:]

print('overall test size')
print(X_test.shape)
test_set = TestObject(X_test)

set_random_seeds(seed=42, cuda=cuda)
input_time_length = X_test.shape[2]
print(input_time_length)
in_chans=X_test.shape[1]
labelsize=3
model = util.shallow_net.EEGShallowClassifier(in_chans, labelsize, input_time_length, return_feature=False)
if cuda:
    model.cuda()


if cuda:
    model.cuda()
checkpoint = torch.load('D:\\motor_imagery\\MImodelinLeaderboard\\'+'cnn_model_MI_'+pilotname+'.pth')
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

average_acc=[]
average_loss=[]
setname='testset'
dataset=test_set

i_trials_in_batch = get_balanced_batches(len(dataset.X), rng, shuffle=False,
                                    batch_size=30)
outputs=None
for i_trials in i_trials_in_batch:
    # Have to add empty fourth dimension to X
#             print(i_trials)
    batch_X = dataset.X[i_trials][:,:,:,None]
#     batch_y = dataset.y[i_trials]
    net_in = np_to_var(batch_X)
    if cuda:
        net_in = net_in.cuda()
    toutputs = model(net_in)
    if outputs is None:
        temp=toutputs.cpu()
        outputs=temp.detach().numpy()
    else:
        temp=toutputs.cpu()
        outputs=np.concatenate((outputs,temp.detach().numpy()))
predicted_labels = np.argmax((outputs), axis=1)
print(predicted_labels.shape)
MI_s2 = predicted_labels
predicted_labels

run 6
(20, 63, 2000)
run 7
(40, 63, 2000)
run 8
(60, 63, 2000)
run 9
(80, 63, 2000)
run 10
(100, 63, 2000)
run 11
(120, 63, 2000)
run 12
(140, 63, 2000)
run 13
(160, 63, 2000)
run 14
(180, 63, 2000)
run 15
(200, 63, 2000)
overall test size
(200, 22, 2000)
2000
(200,)


array([1, 1, 2, 1, 0, 1, 0, 0, 2, 2, 0, 0, 1, 0, 0, 0, 2, 2, 2, 1, 1, 2,
       0, 2, 0, 1, 2, 0, 1, 2, 2, 0, 2, 2, 2, 0, 1, 0, 2, 1, 0, 2, 0, 2,
       1, 2, 2, 0, 1, 0, 0, 1, 0, 0, 2, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0,
       0, 0, 0, 2, 0, 1, 2, 1, 2, 1, 0, 0, 0, 1, 1, 2, 1, 1, 1, 0, 1, 2,
       0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 1, 1, 0, 2, 1, 1, 2, 1, 1, 0, 1, 1,
       0, 1, 2, 1, 2, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 2, 1, 1, 0, 1, 2,
       1, 1, 1, 2, 1, 2, 1, 0, 0, 0, 1, 1, 1, 2, 1, 1, 0, 1, 2, 1, 1, 0,
       0, 1, 1, 0, 1, 0, 0, 2, 1, 1, 1, 1, 1, 1, 0, 0, 0, 2, 1, 1, 0, 0,
       1, 0, 1, 1, 2, 1, 0, 1, 1, 0, 2, 0, 2, 0, 0, 1, 1, 0, 2, 0, 1, 2,
       0, 1], dtype=int64)

In [ ]:
pilotname='3'
savebase0='D:\\leaderboardData\\leaderboardMI\\'
savebase = savebase0+'S'+pilotname+'\\testing\\'

# read crop data
ch_names =['Fp1', 'Fp2', 'F3', 
            'Fz', 'F4', 'FC5', 'FC1', 'FC2','FC6', 'C5', 'C3',
           'C1', 'Cz', 'C2', 'C4', 'C6', 'CP5', 'CP3', 'CP1',
           'CPz', 'CP2', 'CP4', 'CP6', 'P7', 'P5', 'P3', 'P1', 'Pz', 
           'P2', 'P4', 'P6', 'P8']


with open (savebase+"testing_s"+pilotname+"X.npy", 'rb') as fp:
        X_test = pickle.load(fp)

print('overall test size')
print(X_test.shape)
test_set = TestObject(X_test)

set_random_seeds(seed=42, cuda=cuda)
input_time_length = X_test.shape[2]
print(input_time_length)
in_chans=X_test.shape[1]
labelsize=3
model = util.shallow_net.EEGShallowClassifier(in_chans, labelsize, input_time_length, return_feature=False)

if cuda:
    model.cuda()
checkpoint = torch.load('D:\\motor_imagery\\MImodelinLeaderboard\\'+'cnn_model_MI_'+pilotname+'.pth')
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

average_acc=[]
average_loss=[]
setname='testset'
dataset=test_set

i_trials_in_batch = get_balanced_batches(len(dataset.X), rng, shuffle=False,
                                    batch_size=30)
outputs=None
for i_trials in i_trials_in_batch:
    # Have to add empty fourth dimension to X
#             print(i_trials)
    batch_X = dataset.X[i_trials][:,:,:,None]
#     batch_y = dataset.y[i_trials]
    net_in = np_to_var(batch_X)
    if cuda:
        net_in = net_in.cuda()
    toutputs = model(net_in)
    if outputs is None:
        temp=toutputs.cpu()
        outputs=temp.detach().numpy()
    else:
        temp=toutputs.cpu()
        outputs=np.concatenate((outputs,temp.detach().numpy()))
predicted_labels = np.argmax((outputs), axis=1)
print(predicted_labels.shape)
MI_s3 = predicted_labels
predicted_labels

overall test size
(200, 32, 800)
800
(200,)


array([2, 0, 2, 0, 1, 2, 1, 1, 2, 2, 2, 2, 1, 1, 2, 2, 0, 2, 1, 0, 2, 2,
       2, 2, 0, 2, 1, 1, 2, 2, 2, 2, 1, 0, 2, 1, 2, 2, 2, 2, 1, 2, 1, 2,
       1, 2, 2, 2, 1, 0, 2, 2, 1, 2, 1, 2, 0, 1, 1, 2, 2, 2, 1, 2, 2, 2,
       1, 2, 1, 2, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 1, 2, 2,
       0, 0, 2, 2, 1, 2, 1, 2, 1, 2, 2, 1, 2, 2, 1, 2, 1, 1, 2, 0, 1, 2,
       1, 1, 2, 2, 2, 2, 2, 2, 1, 0, 2, 2, 1, 0, 2, 1, 2, 1, 2, 2, 2, 1,
       1, 2, 1, 2, 1, 1, 1, 2, 2, 1, 2, 1, 1, 2, 2, 2, 1, 2, 0, 2, 1, 0,
       1, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 0, 0, 2, 2, 1, 1, 0, 2, 0, 0, 2,
       1, 2, 2, 1, 1, 1, 2, 2, 1, 1, 2, 2, 2, 0, 2, 2, 1, 2, 2, 2, 2, 2,
       1, 1], dtype=int64)

In [ ]:
pilotname='4'
savebase0='D:\\leaderboardData\\leaderboardMI\\'
savebase = savebase0+'S'+pilotname+'\\testing\\'

# read crop data
ch_names =['Fp1', 'Fp2', 'F3', 
            'Fz', 'F4', 'FC5', 'FC1', 'FC2','FC6', 'C5', 'C3',
           'C1', 'Cz', 'C2', 'C4', 'C6', 'CP5', 'CP3', 'CP1',
           'CPz', 'CP2', 'CP4', 'CP6', 'P7', 'P5', 'P3', 'P1', 'Pz', 
           'P2', 'P4', 'P6', 'P8']


with open (savebase+"testing_s"+pilotname+"X.npy", 'rb') as fp:
        X_test = pickle.load(fp)

print('overall test size')
print(X_test.shape)
test_set = TestObject(X_test)

set_random_seeds(seed=42, cuda=cuda)
input_time_length = X_test.shape[2]
print(input_time_length)
in_chans=X_test.shape[1]
labelsize=3
model = util.shallow_net.EEGShallowClassifier(in_chans, labelsize, input_time_length, return_feature=False)

if cuda:
    model.cuda()
checkpoint = torch.load('D:\\motor_imagery\\MImodelinLeaderboard\\'+'cnn_model_MI_'+pilotname+'.pth')
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

average_acc=[]
average_loss=[]
setname='testset'
dataset=test_set

i_trials_in_batch = get_balanced_batches(len(dataset.X), rng, shuffle=False,
                                    batch_size=30)
outputs=None
for i_trials in i_trials_in_batch:
    # Have to add empty fourth dimension to X
#             print(i_trials)
    batch_X = dataset.X[i_trials][:,:,:,None]
#     batch_y = dataset.y[i_trials]
    net_in = np_to_var(batch_X)
    if cuda:
        net_in = net_in.cuda()
    toutputs = model(net_in)
    if outputs is None:
        temp=toutputs.cpu()
        outputs=temp.detach().numpy()
    else:
        temp=toutputs.cpu()
        outputs=np.concatenate((outputs,temp.detach().numpy()))
predicted_labels = np.argmax((outputs), axis=1)
print(predicted_labels.shape)
MI_s4 = predicted_labels
predicted_labels

overall test size
(200, 32, 800)
800
(200,)


array([1, 2, 0, 2, 2, 2, 0, 2, 2, 0, 1, 2, 2, 2, 2, 2, 1, 2, 0, 1, 2, 0,
       1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 2, 0, 0, 2, 2, 0, 1, 0, 0, 1,
       0, 2, 1, 1, 0, 1, 1, 2, 0, 0, 0, 0, 2, 2, 2, 0, 2, 2, 1, 1, 0, 2,
       0, 2, 1, 0, 0, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 2, 0, 2, 2,
       2, 0, 2, 2, 1, 1, 0, 2, 2, 1, 0, 2, 2, 0, 1, 2, 2, 1, 0, 0, 1, 0,
       2, 1, 2, 1, 1, 0, 0, 1, 2, 0, 2, 0, 2, 2, 0, 0, 2, 0, 0, 2, 2, 0,
       2, 2, 0, 1, 2, 2, 0, 1, 2, 2, 2, 0, 0, 1, 0, 2, 1, 1, 2, 2, 0, 1,
       0, 2, 0, 0, 2, 0, 2, 1, 2, 2, 1, 0, 2, 1, 2, 1, 0, 2, 0, 0, 2, 2,
       0, 0, 2, 0, 0, 0, 1, 0, 2, 0, 2, 1, 2, 2, 2, 2, 0, 2, 0, 0, 2, 1,
       2, 0], dtype=int64)

In [ ]:
pilotname='5'
savebase0='D:\\leaderboardData\\leaderboardMI\\'
savebase = savebase0+'S'+pilotname+'\\testing\\'

# read crop data
ch_names =['Fp1', 'Fp2', 'F3', 
            'Fz', 'F4', 'FC5', 'FC1', 'FC2','FC6', 'C5', 'C3',
           'C1', 'Cz', 'C2', 'C4', 'C6', 'CP5', 'CP3', 'CP1',
           'CPz', 'CP2', 'CP4', 'CP6', 'P7', 'P5', 'P3', 'P1', 'Pz', 
           'P2', 'P4', 'P6', 'P8']


with open (savebase+"testing_s"+pilotname+"X.npy", 'rb') as fp:
        X_test = pickle.load(fp)

print('overall test size')
print(X_test.shape)
test_set = TestObject(X_test)

set_random_seeds(seed=42, cuda=cuda)
input_time_length = X_test.shape[2]
print(input_time_length)
in_chans=X_test.shape[1]
labelsize=3
model = util.shallow_net.EEGShallowClassifier(in_chans, labelsize, input_time_length, return_feature=False)

if cuda:
    model.cuda()
checkpoint = torch.load('D:\\motor_imagery\\MImodelinLeaderboard\\'+'cnn_model_MI_'+pilotname+'.pth')
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

average_acc=[]
average_loss=[]
setname='testset'
dataset=test_set

i_trials_in_batch = get_balanced_batches(len(dataset.X), rng, shuffle=False,
                                    batch_size=30)
outputs=None
for i_trials in i_trials_in_batch:
    # Have to add empty fourth dimension to X
#             print(i_trials)
    batch_X = dataset.X[i_trials][:,:,:,None]
#     batch_y = dataset.y[i_trials]
    net_in = np_to_var(batch_X)
    if cuda:
        net_in = net_in.cuda()
    toutputs = model(net_in)
    if outputs is None:
        temp=toutputs.cpu()
        outputs=temp.detach().numpy()
    else:
        temp=toutputs.cpu()
        outputs=np.concatenate((outputs,temp.detach().numpy()))
predicted_labels = np.argmax((outputs), axis=1)
print(predicted_labels.shape)
MI_s5 = predicted_labels
predicted_labels

overall test size
(200, 32, 800)
800
(200,)


array([2, 1, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 1, 2, 0, 2, 2, 2, 0, 2,
       2, 2, 2, 2, 0, 0, 0, 2, 0, 1, 0, 0, 2, 0, 2, 2, 2, 0, 2, 2, 2, 1,
       2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 0, 1, 2, 0,
       1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 1, 2, 2,
       2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 1, 2, 1, 2, 0, 2, 1, 2, 2, 2,
       2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 1, 2, 2, 2, 0, 0, 2, 2, 2, 0, 2, 2, 2, 2, 1, 1, 2, 2, 2, 1,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 1, 2, 0, 0, 1, 1, 2,
       2, 2, 2, 2, 1, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2, 1,
       2, 2], dtype=int64)

**These will be the predicitons to save in a txt file. Format for text saving will be released in next update of this tutorial together with the release of leaderboard**

In [ ]:
print(sleep_label.shape)
print(MI_s1.shape)
print(MI_s2.shape)
print(MI_s3.shape)
print(MI_s4.shape)
print(MI_s5.shape)

(25748,)
(200,)
(200,)
(200,)
(200,)
(200,)


In [ ]:
print(sleep_label[:20])
#label from 0 - 5
np.savetxt("util/pred_sleep_label.txt",sleep_label,delimiter=',',fmt="%d")

[0 1 0 0 2 2 1 1 2 2 2 0 0 1 1 5 0 2 2 0]


In [ ]:
MI_label = MI_s1
MI_label = np.concatenate((MI_label,MI_s2))
MI_label = np.concatenate((MI_label,MI_s3))
MI_label = np.concatenate((MI_label,MI_s4))
MI_label = np.concatenate((MI_label,MI_s5))
print(MI_label.shape)
MI_label = MI_label.astype(int)
MI_label[:100]
#lable 0,1,2
np.savetxt("util/pred_MI_label.txt",MI_label,delimiter=',',fmt="%d")

(1000,)


**When you upload the txt label file to CodaLab, please rename the txt file as 'answer.txt' and zip it before uploading. The score is computed according to classification accuracies with weights of inverse frequency of a label.**